# XRP Price Forecasting Model

This notebook demonstrates how to use the XRP price forecasting model for cryptocurrency price prediction.

## Overview
- Uses LSTM neural networks for time series prediction
- Incorporates technical indicators (RSI, MACD, Bollinger Bands, etc.)
- Predicts future XRP prices based on historical 3-minute candlestick data

## 1. Setup and Imports

In [ ]:
# Import required libraries
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

# Import our model
from model import XRPPriceForecaster

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All imports successful!")

## 2. Explore the Dataset

In [ ]:
# Load and explore one of the dataset files
dataset_files = [f for f in os.listdir("dataset") if f.endswith(".json")]
print(f"Found {len(dataset_files)} dataset files:")
for file in dataset_files:
    print(f"  - {file}")

# Load the first file as an example
with open(f"dataset/{dataset_files[0]}", 'r') as f:
    sample_data = json.load(f)

print(f"\nSample file structure:")
print(f"- Symbol: {sample_data['metadata']['symbol']}")
print(f"- Interval: {sample_data['metadata']['interval']}")
print(f"- Data points: {sample_data['metadata']['data_count']}")
print(f"- Date range: {sample_data['metadata']['date_range']['start_date']} to {sample_data['metadata']['date_range']['end_date']}")

In [ ]:
# Convert to DataFrame for analysis
df_sample = pd.DataFrame(sample_data['market_data'])
df_sample['timestamp'] = pd.to_datetime(df_sample['timestamp_ms'], unit='ms')
df_sample = df_sample.sort_values('timestamp').reset_index(drop=True)

print("Sample data:")
print(df_sample.head())

print("\nPrice statistics:")
print(df_sample[['open', 'high', 'low', 'close']].describe())

## 3. Visualize the Data

In [ ]:
# Plot price data
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# OHLC prices
axes[0, 0].plot(df_sample['timestamp'], df_sample['close'], label='Close', alpha=0.8)
axes[0, 0].plot(df_sample['timestamp'], df_sample['high'], label='High', alpha=0.6)
axes[0, 0].plot(df_sample['timestamp'], df_sample['low'], label='Low', alpha=0.6)
axes[0, 0].set_title('XRP Price Over Time')
axes[0, 0].set_ylabel('Price (USDT)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Price volatility
df_sample['volatility'] = df_sample['close'].rolling(window=10).std()
axes[0, 1].plot(df_sample['timestamp'], df_sample['volatility'])
axes[0, 1].set_title('Price Volatility (10-period rolling std)')
axes[0, 1].set_ylabel('Volatility')
axes[0, 1].grid(True, alpha=0.3)

# Price changes
df_sample['price_change'] = df_sample['close'].diff()
axes[1, 0].plot(df_sample['timestamp'], df_sample['price_change'])
axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].set_title('Price Changes')
axes[1, 0].set_ylabel('Price Change')
axes[1, 0].grid(True, alpha=0.3)

# Price distribution
axes[1, 1].hist(df_sample['close'], bins=30, alpha=0.7)
axes[1, 1].set_title('Price Distribution')
axes[1, 1].set_xlabel('Price (USDT)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Initialize and Train the Model

In [ ]:
# Initialize the forecaster
forecaster = XRPPriceForecaster(
    sequence_length=20,    # Use 20 previous timesteps (1 hour of 3-min data)
    prediction_hours=1     # Predict 1 hour ahead
)

print("Model initialized with:")
print(f"- Sequence length: {forecaster.sequence_length} timesteps")
print(f"- Prediction horizon: {forecaster.prediction_hours} hour(s)")
print(f"- Prediction steps: {forecaster.prediction_steps} (3-min intervals)")

In [ ]:
# Train the model (this may take several minutes)
print("Starting model training... This may take a few minutes.")

history = forecaster.train(
    dataset_folder="dataset",
    validation_split=0.2,
    epochs=20,  # Reduced for notebook demo - increase for better performance
    batch_size=32
)

print("\n✅ Training completed!")

## 5. Visualize Training Results

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Loss curves
ax1.plot(history.history['loss'], label='Training Loss')
ax1.plot(history.history['val_loss'], label='Validation Loss')
ax1.set_title('Model Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# MAE curves
ax2.plot(history.history['mae'], label='Training MAE')
ax2.plot(history.history['val_mae'], label='Validation MAE')
ax2.set_title('Model MAE')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('MAE')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Make Predictions

In [ ]:
# Use the most recent dataset file for prediction demo
recent_data_file = f"dataset/{sorted(dataset_files)[-1]}"
print(f"Using recent data from: {recent_data_file}")

# Make prediction
prediction_result = forecaster.predict(
    recent_data_file=recent_data_file,
    output_file="notebook_prediction.json"
)

print("\n✅ Prediction completed!")

## 7. Visualize Predictions

In [ ]:
# Load recent data for comparison
with open(recent_data_file, 'r') as f:
    recent_data = json.load(f)

# Convert to DataFrames
df_recent = pd.DataFrame(recent_data['market_data'])
df_recent['timestamp'] = pd.to_datetime(df_recent['timestamp_ms'], unit='ms')
df_recent = df_recent.sort_values('timestamp').reset_index(drop=True)

df_prediction = pd.DataFrame(prediction_result['market_data'])
df_prediction['timestamp'] = pd.to_datetime(df_prediction['timestamp_ms'], unit='ms')

# Plot historical vs predicted prices
plt.figure(figsize=(15, 8))

# Plot recent historical data (last 60 points for better visualization)
recent_subset = df_recent.tail(60)
plt.plot(recent_subset['timestamp'], recent_subset['close'], 
         label='Historical Prices', color='blue', linewidth=2)

# Plot predictions
plt.plot(df_prediction['timestamp'], df_prediction['close'], 
         label='Predicted Prices', color='red', linewidth=2, linestyle='--')

# Add vertical line to separate historical and predicted data
plt.axvline(x=recent_subset['timestamp'].iloc[-1], 
           color='green', linestyle='-', alpha=0.7, label='Prediction Start')

plt.title('XRP Price Prediction: Historical vs Predicted', fontsize=16)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Price (USDT)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print prediction summary
predicted_prices = [float(data['close']) for data in prediction_result['market_data']]
print(f"\n📊 Prediction Summary:")
print(f"- Prediction period: {df_prediction['timestamp'].min()} to {df_prediction['timestamp'].max()}")
print(f"- Number of predictions: {len(predicted_prices)}")
print(f"- Price range: ${min(predicted_prices):.4f} - ${max(predicted_prices):.4f}")
print(f"- Average predicted price: ${np.mean(predicted_prices):.4f}")

# Calculate trend
first_price = predicted_prices[0]
last_price = predicted_prices[-1]
change = last_price - first_price
change_pct = (change / first_price) * 100

trend_direction = "📈 Upward" if change > 0 else "📉 Downward" if change < 0 else "➡️ Sideways"
print(f"- Predicted trend: {trend_direction} ({change_pct:+.2f}%)")

## 8. Save the Trained Model

In [ ]:
# Save the trained model for future use
forecaster.save_model("notebook_xrp_model.h5", "notebook_scaler.pkl")

print("✅ Model saved successfully!")
print("You can now use this trained model for future predictions without retraining.")

## 9. Model Analysis and Insights

In [ ]:
# Analyze prediction characteristics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Prediction price distribution
axes[0, 0].hist(predicted_prices, bins=15, alpha=0.7, color='red')
axes[0, 0].set_title('Distribution of Predicted Prices')
axes[0, 0].set_xlabel('Price (USDT)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Price changes in predictions
pred_changes = np.diff(predicted_prices)
axes[0, 1].plot(pred_changes, marker='o', markersize=4)
axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0, 1].set_title('Predicted Price Changes')
axes[0, 1].set_xlabel('Time Step')
axes[0, 1].set_ylabel('Price Change')
axes[0, 1].grid(True, alpha=0.3)

# Volatility analysis
pred_volatility = np.std(predicted_prices)
recent_volatility = np.std(recent_subset['close'].tail(20))

volatilities = [recent_volatility, pred_volatility]
labels = ['Recent Historical', 'Predicted']
axes[1, 0].bar(labels, volatilities, color=['blue', 'red'], alpha=0.7)
axes[1, 0].set_title('Volatility Comparison')
axes[1, 0].set_ylabel('Price Volatility (Std Dev)')
axes[1, 0].grid(True, alpha=0.3)

# Prediction confidence intervals (simplified)
prediction_mean = np.mean(predicted_prices)
prediction_std = np.std(predicted_prices)
time_steps = range(len(predicted_prices))

axes[1, 1].plot(time_steps, predicted_prices, 'r-', label='Prediction', linewidth=2)
axes[1, 1].fill_between(time_steps, 
                       [p - prediction_std for p in predicted_prices],
                       [p + prediction_std for p in predicted_prices],
                       alpha=0.3, color='red', label='±1 Std Dev')
axes[1, 1].set_title('Prediction with Uncertainty Band')
axes[1, 1].set_xlabel('Time Step')
axes[1, 1].set_ylabel('Price (USDT)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📈 Model Analysis:")
print(f"- Historical volatility: {recent_volatility:.4f}")
print(f"- Predicted volatility: {pred_volatility:.4f}")
print(f"- Volatility ratio: {pred_volatility/recent_volatility:.2f}x")
print(f"- Max predicted change: {max(pred_changes):.4f}")
print(f"- Min predicted change: {min(pred_changes):.4f}")

## 10. Next Steps and Usage

Now that you have a trained model, you can:

1. **Make new predictions** with fresh data:
   ```python
   # Load saved model
   new_forecaster = XRPPriceForecaster()
   new_forecaster.load_model("notebook_xrp_model.h5", "notebook_scaler.pkl")
   
   # Make prediction with new data
   result = new_forecaster.predict("new_recent_data.json", "new_prediction.json")
   ```

2. **Experiment with different parameters**:
   - Increase `sequence_length` for longer historical context
   - Adjust `prediction_hours` for different forecasting horizons
   - Try different training parameters (epochs, batch_size, etc.)

3. **Improve the model**:
   - Add more technical indicators
   - Use ensemble methods
   - Incorporate external market data
   - Implement different neural network architectures

4. **Validate performance**:
   - Backtest on historical data
   - Compare with baseline models
   - Analyze prediction accuracy over time

Remember: This model is for educational purposes. Cryptocurrency trading involves significant risk!